In [171]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn as sk
from sklearn import svm
import warnings

warnings.filterwarnings('ignore')

%matplotlib notebook

In [172]:
train_data = pd.read_table(filepath_or_buffer = 'features.train.txt',
                           delim_whitespace = True,
                           names = ['digit', 'intensity', 'symmetry'],
                           dtype = {'digit': np.int32, 'intensity': np.float64, 'symmetry': np.float64})
test_data = pd.read_table(filepath_or_buffer = 'features.test.txt',
                          delim_whitespace = True,
                          names = ['digit', 'intensity', 'symmetry'],
                          dtype = {'digit': np.int32, 'intensity': np.float64, 'symmetry': np.float64})

In [173]:
train_data.head()

,digit,intensity,symmetry
0,6,0.341092,-4.528937
1,5,0.444131,-5.496812
2,4,0.231002,-2.886750
3,7,0.200275,-3.534375
4,3,0.291936,-4.352062


In [174]:
test_data.head()

,digit,intensity,symmetry
0,9,0.272178,-4.847937
1,6,0.265133,-5.102000
2,3,0.335926,-2.921562
3,6,0.264850,-4.156625
4,6,0.345338,-6.718438


In [359]:
def filter_data(data, n, m = -1):
    if m != -1:
        data = data.query('digit == {} or digit == {}'.format(n, m))
    data['Class'] = data.digit.map(lambda x: 1 if x == n else -1)
    return data

def svm_kernel(train_data,test_data, kernel, C, Q=1):
    svc = svm.SVC(C=C, kernel=kernel, degree=Q, coef0=1.0, gamma=1.0)
    svc.fit(train_data[['intensity', 'symmetry']], train_data.Class)
    e_out = 1 - sk.metrics.accuracy_score(test_data.Class, svc.predict(test_data[['intensity', 'symmetry']]))
    e_in = 1 - sk.metrics.accuracy_score(train_data.Class, svc.predict(train_data[['intensity', 'symmetry']]))
    print("E_in ::", e_in)
    print("E_out ::", e_out)
    print("Number of support vecotors ::", len(svc.support_))

### Solution to question 2:

In [348]:
C = 0.01
Q = 2
for i in range(0, 10, 2):
    print('{} versus all'.format(i))
    svm_kernel(filter_data(train_data, i), filter_data(test_data, i), 'poly', C, Q)
    print()

0 versus all
E_in :: 0.105883966534
E_out :: 0.111609367215
Number of support vecotors :: 2179

2 versus all
E_in :: 0.100260595254
E_out :: 0.0986547085202
Number of support vecotors :: 1970

4 versus all
E_in :: 0.0894253188863
E_out :: 0.0996512207275
Number of support vecotors :: 1856

6 versus all
E_in :: 0.0910711836511
E_out :: 0.0847035376183
Number of support vecotors :: 1893

8 versus all
E_in :: 0.0743382252092
E_out :: 0.0827105132038
Number of support vecotors :: 1776



### Solution to question 3:

In [350]:
C = 0.01
Q = 2

for i in range(1, 10, 2):
    print('{} versus all'.format(i))
    svm_kernel(filter_data(train_data, i), filter_data(test_data, i), 'poly', C, Q)
    print()

1 versus all
E_in :: 0.0144013166918
E_out :: 0.02192326856
Number of support vecotors :: 386

3 versus all
E_in :: 0.0902482512687
E_out :: 0.0827105132038
Number of support vecotors :: 1950

5 versus all
E_in :: 0.0762584007681
E_out :: 0.079720976582
Number of support vecotors :: 1585

7 versus all
E_in :: 0.0884652311068
E_out :: 0.0732436472347
Number of support vecotors :: 1704

9 versus all
E_in :: 0.0883280757098
E_out :: 0.0881913303438
Number of support vecotors :: 1978



### Solution to question 5:

In [352]:
Q = 2
C = [0.001, 0.01, 0.1, 1]

for c in C:
    print('C = {}'.format(c))
    svm_kernel(filter_data(train_data, 1, 5), filter_data(test_data, 1, 5), 'poly', c, Q)
    print()

C = 0.001
E_in :: 0.00448430493274
E_out :: 0.0165094339623
Number of support vecotors :: 76

C = 0.01
E_in :: 0.00448430493274
E_out :: 0.0188679245283
Number of support vecotors :: 34

C = 0.1
E_in :: 0.00448430493274
E_out :: 0.0188679245283
Number of support vecotors :: 24

C = 1
E_in :: 0.00320307495195
E_out :: 0.0188679245283
Number of support vecotors :: 24



### Solution to question 6:

In [354]:
Q = 5
C = [0.001, 0.01, 0.1, 1]

for c in C:
    print('C = {}'.format(c))
    svm_kernel(filter_data(train_data, 1, 5), filter_data(test_data, 1, 5), 'poly', c, Q)
    print()

C = 0.001
E_in :: 0.00448430493274
E_out :: 0.0212264150943
Number of support vecotors :: 25

C = 0.01
E_in :: 0.00384368994234
E_out :: 0.0212264150943
Number of support vecotors :: 23

C = 0.1
E_in :: 0.00320307495195
E_out :: 0.0188679245283
Number of support vecotors :: 25

C = 1
E_in :: 0.00320307495195
E_out :: 0.0212264150943
Number of support vecotors :: 21



### Solution to question 7:

In [344]:
def svm_poly_cv(train_data, c, Q, cv):
    runs = 100
    average = 0
    for _ in range(100):
        svc = svm.SVC(C=c, kernel='poly', degree=Q, coef0=1.0, gamma=1.0)
        ecv = 1 - sk.model_selection.cross_val_score(svc, train_data[['intensity', 'symmetry']], train_data.Class, cv=cv).mean()
        average += ecv
    
    print("C:", c)
    print("E_cv:", average/runs)

In [345]:
Q = 2
C = [0.0001, 0.001, 0.01, 0.1, 1]

for c in C:
    svm_poly_cv(filter_data(train_data, 1, 5), c, Q, 10)
    print()

C: 0.0001
E_cv: 0.00962397201456

C: 0.001
E_cv: 0.00513249882779

C: 0.01
E_cv: 0.00449555615263

C: 0.1
E_cv: 0.00514071744296

C: 1
E_cv: 0.00449555615263



### Solution to question 8:

In [360]:
C = [0.01, 1, 100, 1000, 1000000]

for c in C:
    print('C = {}'.format(c))
    svm_kernel(filter_data(train_data, 1, 5), filter_data(test_data, 1, 5), 'rbf', c)
    print()

C = 0.01
E_in :: 0.00384368994234
E_out :: 0.0235849056604
Number of support vecotors :: 406

C = 1
E_in :: 0.00448430493274
E_out :: 0.0212264150943
Number of support vecotors :: 31

C = 100
E_in :: 0.00320307495195
E_out :: 0.0188679245283
Number of support vecotors :: 22

C = 1000
E_in :: 0.00320307495195
E_out :: 0.0188679245283
Number of support vecotors :: 20

C = 1000000
E_in :: 0.000640614990391
E_out :: 0.0235849056604
Number of support vecotors :: 17

